In [1]:
import os
import json
import scipy.io
import pandas as pd
from pathlib import Path
from helper import matfile_to_dic

In [2]:
dataPath=Path("12k_DE")
dataDic=matfile_to_dic(dataPath)

In [3]:
print(len(dataDic))
print(dataDic)

64
{'B007_0.mat': {'__header__': b'MATLAB 5.0 MAT-file, Platform: PCWIN, Created on: Mon Jan 31 13:55:05 2000', '__version__': '1.0', '__globals__': [], 'X118_DE_time': array([-0.0027614 , -0.09632403,  0.11370459, ..., -0.07829373,
       -0.14911545,  0.02111657]), 'X118_FE_time': array([-0.24716182,  0.14279091,  0.00328727, ...,  0.01643636,
       -0.03842   , -0.16806182]), 'X118_BA_time': array([ 0.01553163,  0.01693994, -0.03645507, ...,  0.03178754,
       -0.03243134, -0.0668343 ]), 'X118RPM': 1796}, 'B007_1.mat': {'__header__': b'MATLAB 5.0 MAT-file, Platform: PCWIN, Created on: Mon Jan 31 13:56:04 2000', '__version__': '1.0', '__globals__': [], 'X119_DE_time': array([-0.0721212 ,  0.30310395,  0.03313677, ...,  0.08820228,
       -0.19687138, -0.03898443]), 'X119_FE_time': array([ 0.07745636,  0.04561091, -0.01109455, ..., -0.01273818,
        0.04170727,  0.02753091]), 'X119_BA_time': array([0.07327228, 0.06932902, 0.05480332, ..., 0.05753947, 0.01569258,
       0.01488783

In [4]:
for dicson in dataDic.values():
    del dicson['__header__']
    del dicson['__version__']
    del dicson['__globals__']
dataDic

{'B007_0.mat': {'X118_DE_time': array([-0.0027614 , -0.09632403,  0.11370459, ..., -0.07829373,
         -0.14911545,  0.02111657]),
  'X118_FE_time': array([-0.24716182,  0.14279091,  0.00328727, ...,  0.01643636,
         -0.03842   , -0.16806182]),
  'X118_BA_time': array([ 0.01553163,  0.01693994, -0.03645507, ...,  0.03178754,
         -0.03243134, -0.0668343 ]),
  'X118RPM': 1796},
 'B007_1.mat': {'X119_DE_time': array([-0.0721212 ,  0.30310395,  0.03313677, ...,  0.08820228,
         -0.19687138, -0.03898443]),
  'X119_FE_time': array([ 0.07745636,  0.04561091, -0.01109455, ..., -0.01273818,
          0.04170727,  0.02753091]),
  'X119_BA_time': array([0.07327228, 0.06932902, 0.05480332, ..., 0.05753947, 0.01569258,
         0.01488783]),
  'X119RPM': 1772},
 'B007_2.mat': {'X120_DE_time': array([-0.31447441,  0.12198878,  0.39146866, ...,  0.01900491,
          0.21067836,  0.0568523 ]),
  'X120_FE_time': array([ 0.14792727, -0.04499455,  0.10806909, ..., -0.02177818,
         

In [5]:
processed_data={0:[],1:[],2:[],3:[]}
#0:B 1:IR 2:No 3:OR
for name,data in dataDic.items():
    if 'B' in name:
        for iname in data.keys():
            if 'DE' in iname:
                processed_data[0].extend(data[iname])
    elif 'IR' in name:
        for iname in data.keys():
            if 'DE' in iname:
                processed_data[1].extend(data[iname])
    elif 'No' in name:
        for iname in data.keys():
            if 'DE' in iname:
                processed_data[2].extend(data[iname])
    elif 'OR' in name:
        for iname in data.keys():
            if 'DE' in iname:
                processed_data[3].extend(data[iname])


In [6]:
print(len(processed_data))
for i in range(4):
    print(len(processed_data[i]))

4
1947286
1948115
2182450
3418357


In [47]:
#舍弃这种方式
processed_data={0:[],1:[],2:[],3:[]}
for name,data in dataDic.items():
    if 'B' in name:
        for iname in data.keys():
            if 'DE' in iname:
                processed_data[0].append(data[iname])
    elif 'IR' in name:
        for iname in data.keys():
            if 'DE' in iname:
                processed_data[1].append(data[iname])
    elif 'No' in name:
        for iname in data.keys():
            if 'DE' in iname:
                processed_data[2].append(data[iname])
    elif 'OR' in name:
        for iname in data.keys():
            if 'DE' in iname:
                processed_data[3].append(data[iname])

In [57]:
#丢弃这一步
min=9999999
def comparemin(x1,x2):
    if x1>x2:
        return x2
    else:
        return x1
for data in processed_data.values():
    for iarray in data:
        ilen=len(iarray)
        min=comparemin(ilen,min)
print(min)
#min是所有文件的DE数据中的最小值

120801


In [ ]:
#这一步去掉不要
pprocessed_data={0:[],1:[],2:[],3:[]}
for name,data in processed_data.items():
    print(name)
    print(len(data))
    for idata in data:
        pprocessed_data[name].extend(idata[0:min])

这一步的目的是保持每个mat文件的采样点的一致。防止在后面进行数据划分的时候丢失某个文件的数据

In [ ]:
#去掉不要
for i in range(len(pprocessed_data)):
    print(len(pprocessed_data[i]))

In [66]:
type(processed_data[0])

list

In [7]:
splited_data=[]
label=[]
for name,data in processed_data.items():
    n=len(data)
    x=n//500
    for i in range(x):
        splited_data.append(data[i*500:(i+1)*500])
        label.append(name)

In [8]:
print(len(label))
print(len(splited_data))

18990
18990
